In [1]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import scipy.stats
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
from nn_resampler import nn_resampler
from phase_est_smc import phase_est_smc

In [3]:
net = nn_resampler(100,100)
#net = net.to(device)
net.load_state_dict(torch.load("net_bn_aft_relu.model"))
net.eval()

nn_resampler(
  (enc1): Linear(in_features=100, out_features=50, bias=True)
  (enc2): Linear(in_features=50, out_features=25, bias=True)
  (enc3): Linear(in_features=25, out_features=10, bias=True)
  (dec1): Linear(in_features=10, out_features=25, bias=True)
  (dec2): Linear(in_features=25, out_features=50, bias=True)
  (dec3): Linear(in_features=50, out_features=100, bias=True)
  (bn_enc1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn_enc2): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn_enc3): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn_dec1): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn_dec2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [4]:
num_particles = 100 # number of SMC particles (num of w points)
num_samples = 10000 # number of samples to draw from the particle distribution (to be binned)
num_bins = 100 # number of bins
n_iters = 100 # number of iterations of resampling

## NN Resampler

In [5]:
omega_star_list = []
nn_pred_list = []

for i in range(n_iters): 
    t0 = 0.1
    omega_star = np.random.uniform() * 2 * np.pi
    omega_star_list.append(omega_star)
    smc = phase_est_smc(omega_star, t0)
    smc.init_particles(num_particles)
    running_est = []

    while True:

        particle_pos, particle_wgts = smc.particles(threshold=num_particles/5, num_measurements=1)
        data = np.random.choice(particle_pos, size = num_samples, p=particle_wgts)
        mean = np.mean(data)
        std = np.std(data)
        data = (data-mean)/std
        bins, edges = np.histogram(data, num_bins)
        bins = bins/num_samples

        nn_pred = net(torch.tensor(bins).float().unsqueeze(0)) ## convert to float tensor, then make dim [1, num_bins]

        smc.nn_resample(nn_pred.detach().numpy(), edges, mean, std)

    #     edges_ = edges[:-1]
    #     edge_width = edges_[1] - edges_[0]

    #     plt.bar(edges_*std + mean, bins, align='edge', width = edge_width)
    #     plt.bar(edges_*std + mean, nn_pred[0].detach().numpy(), align='edge', width = edge_width, alpha=0.7)
    #     plt.show() 
        curr_omega_est = smc.particle_pos[np.argmax(smc.particle_wgts)]
        running_est.append(curr_omega_est)
        
        if len(running_est) > 10:
            last_10 = running_est[-10:]
            # np.allclose(a,b) returns true if every element in a and b are equal to a tolerance
            # np.full_like makes an array with shake of last_10 with every element being last_10[0]
            # if all elements of last_10 are equal then they should all equal the first element
            if np.allclose(last_10, np.full_like(last_10, last_10[0])):
                break
                
    print("True omega: {:f}, prediction by SMC: {:f}".format(omega_star,curr_omega_est))
    nn_pred_list.append(curr_omega_est)


True omega: 5.532031, prediction by SMC: 5.532031
True omega: 6.069187, prediction by SMC: 5.359994
True omega: 4.581181, prediction by SMC: 4.581181
True omega: 0.390340, prediction by SMC: 0.653454
True omega: 5.935103, prediction by SMC: 5.935103
True omega: 6.195464, prediction by SMC: 6.195464
True omega: 4.199959, prediction by SMC: 4.199959
True omega: 3.640833, prediction by SMC: 3.640833
True omega: 3.588375, prediction by SMC: 3.588375
True omega: 0.213626, prediction by SMC: 0.213626
True omega: 2.775813, prediction by SMC: 2.775813
True omega: 6.138574, prediction by SMC: 6.138572
True omega: 5.356662, prediction by SMC: 5.356662
True omega: 3.770662, prediction by SMC: 3.770662
True omega: 3.514182, prediction by SMC: 3.514182
True omega: 0.900441, prediction by SMC: 0.900408
True omega: 5.882085, prediction by SMC: 5.882085
True omega: 5.171723, prediction by SMC: 5.171723
True omega: 2.999521, prediction by SMC: 2.999521
True omega: 3.721778, prediction by SMC: 3.721778


In [6]:
nn_mse = np.mean( (np.array(omega_star_list) - np.array(nn_pred_list))**2 )
print(nn_mse)

0.017417795820513143


## Liu-West resampler

In [7]:
omega_star_list = []
lw_pred_list = []

for i in range(n_iters): 
    t0 = 0.1
    omega_star = np.random.uniform() * 2 * np.pi
    omega_star_list.append(omega_star)
    smc = phase_est_smc(omega_star, t0)
    smc.init_particles(num_particles)
    running_est = []

    while True:

        smc.particles(threshold=num_particles/5, num_measurements=1)
        smc.liu_west_resample()
        
        if smc.lw_break_flag is True:
            break
        
        curr_omega_est = smc.particle_pos[np.argmax(smc.particle_wgts)]
        running_est.append(curr_omega_est)
        
        if len(running_est) > 5:
            last_5 = running_est[-5:]
            # np.allclose(a,b) returns true if every element in a and b are equal to a tolerance
            # np.full_like makes an array with shake of last_10 with every element being last_10[0]
            # if all elements of last_10 are equal then they should all equal the first element
            if np.allclose(last_5, np.full_like(last_5, last_5[0])):
                break
                
    print("True omega: {:f}, prediction by LW: {:f}".format(omega_star,curr_omega_est))
    lw_pred_list.append(curr_omega_est)

True omega: 1.021260, prediction by LW: 1.021260
True omega: 3.595277, prediction by LW: 3.595277
True omega: 0.711463, prediction by LW: 0.711463
True omega: 2.952453, prediction by LW: 2.952453
True omega: 1.740761, prediction by LW: 1.740761
True omega: 4.276262, prediction by LW: 4.276262
True omega: 1.225911, prediction by LW: 1.225911
True omega: 0.980640, prediction by LW: 0.980640
True omega: 1.921498, prediction by LW: 1.921497
True omega: 1.047306, prediction by LW: 1.047306
True omega: 5.318141, prediction by LW: 5.318142
True omega: 0.139604, prediction by LW: 0.139603
True omega: 5.685974, prediction by LW: 5.685974
True omega: 5.115761, prediction by LW: 5.115761
True omega: 2.259481, prediction by LW: 2.259481
True omega: 2.357117, prediction by LW: 2.357117
True omega: 2.322564, prediction by LW: 2.322564
True omega: 6.126307, prediction by LW: 6.126307
True omega: 3.299778, prediction by LW: 3.299778
True omega: 4.152168, prediction by LW: 3.939171
True omega: 1.459916

In [8]:
lw_mse = np.mean( (np.array(omega_star_list) - np.array(lw_pred_list))**2 )
print(lw_mse)

0.00045367616538141253
